# Линейная регрессия на PyTorch

In [52]:
import torch
from torch.utils.data import TensorDataset, DataLoader

%matplotlib inline

In [53]:
num_inputs = 2 # количество признаков (входов)
num_examples = 1000 # количество значений
true_w = torch.tensor([2, -3.4]) # истинные значения весов
true_b = 4.2 # Истинное значение смещения
features = torch.randn((num_examples, num_inputs)) # Генерируем набор значений
labels = torch.mv(features, true_w) + true_b # Рассчитываем метки
labels += torch.randn(labels.shape) # Добавляем к меткам случайный шум

## Dataset - это экземпляр класса TensorDataset

In [54]:
dataset = TensorDataset(features, labels)
print(dataset.tensors[0])
print(dataset.tensors[1])

tensor([[ 1.6169, -0.2811],
        [ 0.1068, -1.7612],
        [-0.5749,  1.4517],
        ...,
        [ 1.0706,  1.2314],
        [-0.5934, -0.9372],
        [ 0.7511, -0.4628]])
tensor([ 7.3603e+00,  1.1383e+01, -1.2258e+00,  2.9441e-01,  9.7091e+00,
         3.4931e+00,  7.8927e+00, -5.8346e-01,  4.0616e+00,  5.7830e+00,
         7.8042e+00,  5.4642e+00,  8.8773e+00,  8.1266e+00,  4.3782e+00,
         3.6732e+00,  5.8331e+00,  2.6155e+00,  7.3060e+00,  3.8338e+00,
         1.9800e+00,  4.9357e+00,  2.5621e+00,  1.3893e+01,  2.2097e+00,
        -1.4028e+00,  1.1808e+01,  1.6647e+00,  4.8142e+00,  8.2210e-01,
         7.7418e+00, -1.2186e+00,  2.4585e-01,  3.6602e+00, -3.5242e-01,
         2.5200e+00,  9.6757e+00,  2.8899e+00, -2.3405e+00,  8.8528e+00,
         6.9093e+00,  7.3208e+00,  2.7313e+00,  7.8064e+00,  4.9583e-01,
         6.0899e+00,  9.5722e+00,  6.5317e+00,  7.2023e+00,  7.7666e+00,
        -5.7642e-01, -3.1204e-01, -2.1728e+00,  2.1046e+00,  5.2765e+00,
         3.0909

## DataIterator - это экземпляр класса DataLoader

In [55]:
# Randomly reading mini-batches
batch_size = 10
data_iter = DataLoader(dataset, batch_size, shuffle=True)

### Выведем один batch размером = 10

In [56]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-1.1564,  1.4867],
        [ 1.4812,  2.5007],
        [ 0.7936,  0.9376],
        [ 0.9252, -0.8306],
        [ 1.2141, -0.0881],
        [ 0.5697,  0.4753],
        [-0.8011,  0.6381],
        [-0.4970, -1.0039],
        [ 0.8167,  0.4431],
        [-1.2180, -0.1619]]) tensor([-3.6283, -1.4134,  2.7628,  9.3272,  5.4634,  2.9505,  0.2876,  6.4353,
         5.3029,  3.3107])


## Строим модель

### Задаем модель

In [57]:
# var_1 (Позволяет делать мнгослойные модели)
model_01 = torch.nn.Sequential(torch.nn.Linear(2, 1))
model_01

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [58]:
# var_2 (Достаточен для нашего примера)
model_02 = torch.nn.Linear(2, 1)
model_02

Linear(in_features=2, out_features=1, bias=True)

### Инициализируем веса модели

In [59]:
# # var_1
# model_01[0].weight.data = true_w.clone().detach().requires_grad_(True).reshape((1, 2))
# model_01[0].bias.data = torch.tensor([true_b], requires_grad=True)

In [60]:
# # var_2
# model_02.weight.data = true_w.clone().detach().requires_grad_(True).reshape((1, 2))
# model_02.bias.data = torch.tensor([true_b], requires_grad=True)

> По умолчанию Torch инициализирует начальные значения параметров по методу, признанному лучним на сегодняшний день

### Функция потерь

In [61]:
loss = torch.nn.MSELoss(reduction='mean')

### Создаем опримизатор

In [62]:
# var_1
trainer_01 = torch.optim.SGD(model_01.parameters(), lr=0.001)

# var_2
trainer_02 = torch.optim.SGD(model_02.parameters(), lr=0.001)

> Надо всегда следить за размерностями тензоров!

### Обучение

In [63]:
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer_01.zero_grad()
        l = loss(model_01(X).reshape(-1), y)
        l.backward()
        trainer_01.step()
    l = loss(model_01(features).reshape(-1), labels)
    if epoch % 5 == 0:
        print('epoch %d, loss: %f' % (epoch, l.item()),
              '|\tw', model_01[0].weight.data,
              '|\tb', model_01[0].bias.data)    

epoch 5, loss: 4.568942 |	w tensor([[ 1.3520, -2.3620]]) |	b tensor([2.7493])
epoch 10, loss: 1.441925 |	w tensor([[ 1.7501, -3.0129]]) |	b tensor([3.6742])
epoch 15, loss: 1.051614 |	w tensor([[ 1.8944, -3.2383]]) |	b tensor([4.0023])
epoch 20, loss: 1.002722 |	w tensor([[ 1.9461, -3.3186]]) |	b tensor([4.1180])
epoch 25, loss: 0.996641 |	w tensor([[ 1.9652, -3.3475]]) |	b tensor([4.1580])
epoch 30, loss: 0.995881 |	w tensor([[ 1.9713, -3.3568]]) |	b tensor([4.1729])
epoch 35, loss: 0.995788 |	w tensor([[ 1.9745, -3.3594]]) |	b tensor([4.1776])
epoch 40, loss: 0.995772 |	w tensor([[ 1.9758, -3.3612]]) |	b tensor([4.1798])
epoch 45, loss: 0.995774 |	w tensor([[ 1.9775, -3.3609]]) |	b tensor([4.1805])
epoch 50, loss: 0.995772 |	w tensor([[ 1.9770, -3.3623]]) |	b tensor([4.1806])
epoch 55, loss: 0.995770 |	w tensor([[ 1.9764, -3.3615]]) |	b tensor([4.1811])
epoch 60, loss: 0.995770 |	w tensor([[ 1.9758, -3.3618]]) |	b tensor([4.1813])
epoch 65, loss: 0.995770 |	w tensor([[ 1.9753, -3.361

In [64]:
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer_02.zero_grad()
        l = loss(model_02(X).reshape(-1), y)
        l.backward()
        trainer_02.step()
    l = loss(model_02(features).reshape(-1), labels)
    if epoch % 5 == 0:
        print('epoch %d, loss: %f' % (epoch, l.item()),
              '|\tw', model_02.weight.data,
              '|\tb', model_02.bias.data)  

epoch 5, loss: 6.231633 |	w tensor([[ 1.2635, -2.0707]]) |	b tensor([2.4895])
epoch 10, loss: 1.647451 |	w tensor([[ 1.7165, -2.9088]]) |	b tensor([3.5856])
epoch 15, loss: 1.076682 |	w tensor([[ 1.8821, -3.2029]]) |	b tensor([3.9716])
epoch 20, loss: 1.005835 |	w tensor([[ 1.9414, -3.3064]]) |	b tensor([4.1071])
epoch 25, loss: 0.997034 |	w tensor([[ 1.9624, -3.3426]]) |	b tensor([4.1550])
epoch 30, loss: 0.995926 |	w tensor([[ 1.9710, -3.3554]]) |	b tensor([4.1715])
epoch 35, loss: 0.995794 |	w tensor([[ 1.9743, -3.3595]]) |	b tensor([4.1768])
epoch 40, loss: 0.995775 |	w tensor([[ 1.9758, -3.3600]]) |	b tensor([4.1798])
epoch 45, loss: 0.995771 |	w tensor([[ 1.9764, -3.3613]]) |	b tensor([4.1804])
epoch 50, loss: 0.995770 |	w tensor([[ 1.9755, -3.3612]]) |	b tensor([4.1804])
epoch 55, loss: 0.995770 |	w tensor([[ 1.9756, -3.3620]]) |	b tensor([4.1811])
epoch 60, loss: 0.995771 |	w tensor([[ 1.9752, -3.3628]]) |	b tensor([4.1808])
epoch 65, loss: 0.995770 |	w tensor([[ 1.9763, -3.361